In [4]:
# Data ingestion
from langchain_community.document_loaders import TextLoader
loader=TextLoader("speech.txt")
text_document=loader.load()
text_document

[Document(page_content='Ladies and gentlemen, esteemed colleagues,\n\nToday, we stand at the precipice of innovation, where the convergence of technology and human ingenuity promises to redefine the very fabric of our existence. In a world teeming with challenges, we find ourselves armed with the tools of progress, poised to embark on a journey of unprecedented discovery and transformation.\n\nAs we navigate the complexities of the modern era, let us remember that our greatest asset lies not merely in our intellect, but in our capacity to collaborate and innovate. It is through unity and shared vision that we shall surmount the obstacles that lie ahead and unlock the boundless potential of tomorrow.\n\nTogether, we have the power to shape the course of history, to illuminate the darkest corners of ignorance with the light of knowledge, and to build a future where opportunity knows no bounds. Let us seize this moment with unwavering resolve, for it is in our hands that the destiny of ge

In [24]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]='true'
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 5


In [8]:
# web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4

# load,chunk,index the content of the html page
loader=WebBaseLoader(web_path='https://lilianweng.github.io/',
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=('entry-header','entry-content')
                     )),)
text_document=loader.load()
text_document

[Document(page_content='\n👋 Welcome to Lil’Log\n\nHi, this is Lilian. I’m documenting my learning notes in this blog. Besides, I’m leading a team working on practical AI safety and alignment at OpenAI. Based on the number of grammar mistakes in my posts, you can tell how much ChatGPT is involved 😉.\n\nDiffusion Models for Video Generation\n    \n\nDiffusion models have demonstrated strong results on image synthesis in past years. Now the research community has started working on a harder task—using it for video generation. The task itself is a superset of the image case, since an image is a video of 1 frame, and it is much more challenging because:\nIt has extra requirements on temporal consistency across frames in time, which naturally demands more world knowledge to be encoded into the model....\n\nThinking about High-Quality Human Data\n    \n\n[Special thank you to Ian Kivlichan for many useful pointers (E.g. the 100+ year old Nature paper “Vox populi”) and nice feedback. 🙏 ]\nHigh

In [11]:
# Pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('CONSTITUTION.pdf')
doc=loader.load()
doc


[Document(page_content='   \n1 \n CONSTITUTION  \nIntroduction: - The I ndian con stitution is unique in its content and spirit. The salient features of \nthe constitution are as follows: - \n\uf0b7 Lengthiest written Constitution  \n\uf0b7 Blend of Rigidity and Flexibility  \n\uf0b7 Federal system with unitary features  \n\uf0b7 Parliamentary form of Government  \n\uf0b7 Independent judiciary  \n\uf0b7 Single citizenship  \n\uf0b7 Emergency provision  \nStructure: - The Indian Constitution originally consisted of 395 Art, 22 parts, 8 Schedules.  But \nafter the Constitution 104th  Amendment Act, 2003, the Indian Con stitution Consists of 44 8 \nArt, 2 5 parts, 12 Schedules.  \nPreamble: - The preamble to the constitution is based on the “ objective resolution”  drafted and \nmoved by Pandit Nehru and adopted by constituent assembly. It runs as follows: - \n“We  THE PEOPLE OF INDIA,  having solemnly resolved to co nstitute India in to a SOVERIGN \nSOCIALIST  SECULAR  DEMOCRATIC REPUBLI

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(doc)
documents[:5][1]

Document(page_content='“We  THE PEOPLE OF INDIA,  having solemnly resolved to co nstitute India in to a SOVERIGN \nSOCIALIST  SECULAR  DEMOCRATIC REPUBLIC and to secure to all its citizen: - \n\uf0b7 JUSTICE, social, Economic, and Political;  \n\uf0b7 LIBERTY of thought, expression, belief, faith and worship;  \n\uf0b7 EQUALITY of status and of opportunity ;, \n\uf0b7 FRATERNITY  assuring  the dignity  of the individual  and the unity  and integrity  of the \nnation;  In our Constituent  Assembly,  this 26th  November,  1949 , do hereby  adopt,  enact  \nand give to ourselves  this constitution. ” \nPurpose of the Preamble: - The preamble to the constitution is a key to open the minds of \nthe makers and shows the general purpose for which they made the several provisions in \nthe constitution. Preamble serves the following purposes: - \n1. It discloses the source of the const itution . \n2. It lays down the  date of the commencement of the constitution.', metadata={'source': 'CONSTITU

In [26]:
# Vector embedding and Vector store
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
db=Chroma.from_documents(documents[:10],OpenAIEmbeddings())

In [27]:
# Vector database
query="What is the fundamental rights"
result=db.similarity_search(query)
result[0].page_content

'o When  the state  is having  deep  and pervasive  control  over  the affairs  of the \ncorporation.  \no If the corporation  is discharging  public  function.  \n\uf0b7 Corporation  acting  as an agenc y instrumentality  of the government.  ( RD Shetty  vs. IAA).  \n\uf0b7 Stock  Exchange  is not a state  because  it is independent  from  the government  control.  \n(Satish  nayak  vs. Cochin  Stock  Exchange  Ltd.)  \n\uf0b7  \n \nFundamental  Right  \nIntroduction: - The aim of Fundamental Rights is that certain elementary rights such as right to \nlife, liberty, freedom of speech and freedom of faith and so on should be regarded as \ninviolable under all circumstances and that the shifting majority in legislatures of the  country \nshould not have a free hand in interfering with fundamental rights. Fundamental right is \ncalled the Magna Carta of India.  \n   \nRights to Equality  \na. Equality before law -Art 14.  \nb. Prohibition of discrimination on the grounds of religion race

In [52]:
# from langchain_community.llms import ollama
# llm1=ollama(model="llama2")
from openai import OpenAI
from langchain_openai import ChatOpenAI
# llm=OpenAI()
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

Chain and Retriever 


In [49]:
loader=PyPDFLoader('CONSTITUTION.pdf')
doc=loader.load()
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
<context>
{context}
Question: {input}""")

In [57]:
##Chain introduction
##Create stuff document chain
import openai
from langchain.chains.combine_documents import create_stuff_documents_chain

doc_chain=create_stuff_documents_chain(llm,prompt)
retriever=db.as_retriever()
retriever

from langchain.chains import create_retrieval_chain
retrievel_chain=create_retrieval_chain(retriever,doc_chain)
response=retrievel_chain.invoke({"input":"Fundamental rights"})
response['answer']

'Fundamental rights are certain elementary rights such as the right to life, liberty, freedom of speech, freedom of faith, and more that are regarded as inviolable under all circumstances. They aim to ensure that the shifting majority in legislatures of the country do not have a free hand in interfering with these rights. Fundamental rights are often referred to as the Magna Carta of India.'

Multisearch Agent RAG